# Segmenting satellite imagery from the Maxar Open Data Program


This notebook shows how to segment satellite imagery from the [Maxar Open Data](https://github.com/opengeos/maxar-open-data) program for Libya floods.  

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

## Enable Jupyter Lab Leaflet

Run the following cell and reload  the page to enable the Jupyter Lab Leaflet extension.

In [ ]:
!jupyter labextension enable jupyter-leaflet

In [ ]:
import os
import leafmap
from samgeo import SamGeo, raster_to_vector, overlay_images

## Download sample data

First, let's download a sample image of Derna, Libya from [here](https://drive.google.com/file/d/1jIIC5hvSPeJEC0fbDhtxVWk2XV9AxsQD/view?usp=sharing).

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/raster/Derna_sample.tif"

In [ ]:
leafmap.download_file(url, output="image.tif")

## Create an interactive map

In [ ]:
m = leafmap.Map(height="600px")
m.add_basemap("SATELLITE")
m.add_raster("image.tif", layer_name="Image")
m.add_layer_manager()
m

## Initialize SAM class

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [ ]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 80,
}

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    sam_kwargs=sam_kwargs,
)

## Segment the image

In [ ]:
sam.generate("image.tif", output="mask.tif", foreground=True)

## Convert raster to vector

In [ ]:
raster_to_vector("mask.tif", output="mask.shp")

## Display the segmentation result

First, let's show the result as a binary image.

In [ ]:
sam.show_masks(cmap="binary_r")

Display the annotations (each mask with a random color).

In [ ]:
sam.show_anns(axis="off", opacity=1, output="annotation.tif")

## Compare images with a slider

In [ ]:
leafmap.image_comparison(
    "image.tif",
    "annotation.tif",
    label1="Image",
    label2="Segmentation",
)

Overlay the annotations on the image and use the slider to change the opacity interactively.

In [ ]:
overlay_images("image.tif", "annotation.tif", backend="TkAgg")

## Display images on an interactive map.

In [ ]:
m.add_raster("mask.tif", layer_name="Mask", nodata=0)
m.add_raster("annotation.tif", layer_name="Annotation")
m

In [ ]:
m.add_vector("mask.shp", layer_name="Vector", info_mode=None)

![](https://i.imgur.com/kjMi8AF.png)